In [2]:
import pandas as pd
import openpyxl as pyx


# wb = pyx.load_workbook('../sample_data/FHN_pkg_analyzer_example.xlsx')

# print(wb.sheetnames)






In [2]:
# sheet = wb.active

# [cell.value.replace('\n','').strip() for cell in sheet[13] if cell.value != None]

In [8]:
test = pd.read_csv('../sample_data/SAMPLE_FHN.csv').head()

In [ ]:
cols = test.columns.to_list()
cols = [c.strip().replace('\n'," ") for c in cols]
cols = [c for c in test.columns.to_list() if "unnamed" not in str(c).lower()]
test = test[[c for c in test.columns.to_list() if "unnamed" not in str(c).lower()]]

test.columns = cols
test

In [9]:
import pandas as pd
import numpy as np
import openpyxl as pyxl
from dataclasses import dataclass
import json
from pathlib import Path


class LoanTape:
    df: pd.DataFrame
    raw_dfs: list

    def rm_unnamed(self, _cols:list)->list:
        return [c for c in _cols if "unnamed" not in str(c).lower()]

    def norm_raw_cols(self)->None:
        """Normalize the columns -- remove white space and line breaks"""
        if len(self.raw_dfs)>0:
            norm_dfs = []
            for df in self.raw_dfs:
                cols = df.columns.to_list()
                cols = [c.strip().replace("\n"," ") for c in cols]
                df.columns = cols
                norm_dfs.append(df)
            self.raw_dfs = norm_dfs
        else:
            print("No raw loan data")        
        return None
    
    def parse_data(self):
        """parse the raw loan data by using the available packages"""
        pkg_path = Path('package_maps/packages.json')
        pkg_file = open(pkg_path)
        format_opts = json.load(pkg_file)
        return format_opts

    def __init__(self, clean_columns, data=list()):
        self.df = pd.DataFrame(columns=clean_columns)
        self.raw_dfs = data
        self.norm_raw_cols()
        self.raw_dfs = [df[self.rm_unnamed(df.columns.to_list())] for df in self.raw_dfs]

In [15]:
import json
from pathlib import Path

clean_columns= ['Pck / Deal','GP#', 'Days', 'Category', 'Borrower Name', 'City', 'State', 'SIC /NAICS', 'ADJ', 'Accrual', 'Note Date',
'Note Maturity', 'Int. Paid to Date', 'Loan Spread', 'Loan Rate',
'Strip Rate', 'Original Balance', 'Current Balance', 'Multiple', 
'Proceeds', 'Term', 'Age', 'Rmos', 'Industry', 'Prepayment Penalty',
'Term Bucket', 'Industry Bucket', 'Lender', 'Prepayment Notice']

pkg_path = Path('package_maps/packages.json')
pkg_file = open(pkg_path)
pkgs = json.load(pkg_file)

test_FHN = pd.read_csv('../sample_data/SAMPLE_FHN.csv').head()

lt = LoanTape(clean_columns=clean_columns,data=[test_FHN])



test = lt.raw_dfs.pop()
fmt = lt.parse_data()


for key in fmt.keys():
    print(fmt[key])
# for c in test.columns:



{'FHN': {'Trans': '', 'GP#': 'GP#', 'Status': '', 'Inv Status': '', 'Pkg ID': '', 'GTD Balance': 'Current Balance', 'Orig Fee': 'Strip Rate', 'Note Dt': 'Note Date', 'Loan Age': '', 'Matures': 'Note Maturity', 'Orig Mos': '', 'Mat Mos': '', 'Paid To': '', 'Sell Ratio': 'Multiple', 'Sell Value': '', 'Borrower': 'Borrower Name', 'Street Addr': '', 'City/State': 'func_geosplit', 'NAICS': 'SIC / NAICS', 'Gross Cpn': 'Loan Spread', 'Adjust': 'ADJ', 'Pays': '', 'Net Cap': '', 'Net Flr': '', 'Basis': 'Accrual', 'Orig Lender': 'Lender', 'Cert': ''}}
{'Trans': '', 'GP#': 'GP#', 'Status': '', 'Inv Status': '', 'Pkg ID': '', 'GTD Balance': 'Current Balance', 'Orig Fee': 'Strip Rate', 'Note Dt': 'Note Date', 'Loan Age': '', 'Matures': 'Note Maturity', 'Orig Mos': '', 'Mat Mos': '', 'Paid To': '', 'Sell Ratio': 'Multiple', 'Sell Value': '', 'Borrower': 'Borrower Name', 'Street Addr': '', 'City/State': 'func_geosplit', 'NAICS': 'SIC / NAICS', 'Gross Cpn': 'Loan Spread', 'Adjust': 'ADJ', 'Pays': '', 

In [13]:
lt.raw_dfs

[]

In [ ]:
def geo_split(**kwargs):
    print(kwargs.keys())


geo_split(format='FHN')

dict_keys(['format'])
